In [13]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [14]:
# read data from the silver tables
hospital = spark.table("PL7G1_SILVER.hospital")


In [15]:
from pyspark.sql.functions import lower, col

hospital = hospital.withColumn('apr_DRG_Description', lower(col('apr_DRG_Description')))

hospital = hospital.groupBy("hospital_County","discharge_Year").pivot("apr_DRG_Description").count()

hospital = hospital.withColumnRenamed("hospital_County", "borough")
hospital = hospital.withColumnRenamed("discharge_Year", "year")

hospital = hospital.na.fill(value=0)

hospital = hospital.withColumn("bronchitis", col("acute bronchitis & related symptoms") + col("acute bronchitis and related symptoms"))
hospital = hospital.drop("acute bronchitis & related symptoms")
hospital = hospital.drop("acute bronchitis and related symptoms")

hospital = hospital.withColumnRenamed("bronchiolitis & rsv pneumonia", "bronchiolitis")

hospital = hospital.drop("allogeneic bone marrow transplant")
hospital = hospital.drop("autologous bone marrow transplant")
hospital = hospital.drop("autologous bone marrow transplant or t-cell immunotherapy")
hospital = hospital.drop("bone marrow transplant")


hospital = hospital.withColumn("respiratory system diagnosis", col("other respiratory & chest procedures") + col("other respiratory diagnoses except signs, symptoms & minor diagnoses") + col("respiratory system diagnosis w ventilator support 96+ hours") + col("bpd & oth chronic respiratory diseases arising in perinatal period") + col("other respiratory diagnoses except signs, symptoms & minor diagnoses") + col("respiratory system diagnosis w ventilator support 96+ hours") + col("respiratory signs, symptoms & minor diagnoses"))
hospital = hospital.drop("respiratory system diagnose")
hospital = hospital.drop("extensive procedure unrelated to principal diagnosis")
hospital = hospital.drop("other respiratory diagnoses except signs, symptoms & minor diagnoses")
hospital = hospital.drop("respiratory system diagnosis w ventilator support 96+ hours")
hospital = hospital.drop("bpd & oth chronic respiratory diseases arising in perinatal period")
hospital = hospital.drop("other respiratory diagnoses except signs, symptoms & minor diagnoses")
hospital = hospital.drop("respiratory system diagnosis w ventilator support 96+ hours")
hospital = hospital.drop("respiratory signs, symptoms & minor diagnoses")
hospital = hospital.drop("other respiratory & chest procedures")
                               
hospital = hospital.withColumnRenamed("chronic obstructive pulmonary disease", "chronic pulmonary disease")

hospital = hospital.withColumnRenamed("cystic fibrosis - pulmonary disease", "cystic fibrosis")
hospital = hospital.withColumnRenamed("extracorporeal membrane oxygenation (ecmo)", "ECMO")

hospital = hospital.withColumn("tracheostomy with mv", col("tracheostomy w mv 96+ hours w extensive procedure") + col("tracheostomy w mv 96+ hours w extensive procedure or ecmo"))
hospital = hospital.drop("tracheostomy w mv 96+ hours w extensive procedure")
hospital = hospital.drop("tracheostomy w mv 96+ hours w extensive procedure or ecmo")
hospital = hospital.drop("heart &/or lung transplant")
hospital = hospital.withColumnRenamed("interstitial & alveolar lung diseases", "alveolar lung diseases")

hospital = hospital.withColumn("tracheostomy", col("tracheostomy with mv") + col("tracheostomy w mv 96+ hours w/o extensive procedure"))
hospital = hospital.drop("tracheostomy with mv")
hospital = hospital.drop("tracheostomy w mv 96+ hours w/o extensive procedure")
hospital = hospital.drop("moderately extensive procedure unrelated to principal diagnosis")
hospital = hospital.drop("nonextensive procedure unrelated to principal diagnosis")

hospital = hospital.drop("liver transplant &/or intestinal transplant")

hospital = hospital.withColumnRenamed("other pneumonia", "pneumonia")
                               
hospital = hospital.withColumn("respiratory infection", col("major chest & respiratory trauma") + col("major respiratory & chest procedures") + col("major respiratory infections & inflammations"))
hospital = hospital.drop("major chest & respiratory trauma")
hospital = hospital.drop("major respiratory & chest procedures")    
hospital = hospital.drop("major respiratory infections & inflammations")

    
    
hospital = hospital.withColumn('Total', col("asthma") +col("respiratory system diagnosis") + col("bronchiolitis") + col("chronic pulmonary disease") + col("cystic fibrosis") + col("ECMO") + col("alveolar lung diseases") + col("respiratory infection") + col("pneumonia") + col("pulmonary embolism") + col("respiratory failure") + col("respiratory malignancy") + col("bronchitis") + col("tracheostomy"))

In [16]:
hospital.printSchema()

root
 |-- borough: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- asthma: long (nullable = true)
 |-- bronchiolitis: long (nullable = true)
 |-- chronic pulmonary disease: long (nullable = true)
 |-- cystic fibrosis: long (nullable = true)
 |-- ECMO: long (nullable = true)
 |-- alveolar lung diseases: long (nullable = true)
 |-- pneumonia: long (nullable = true)
 |-- pulmonary embolism: long (nullable = true)
 |-- respiratory failure: long (nullable = true)
 |-- respiratory malignancy: long (nullable = true)
 |-- bronchitis: long (nullable = true)
 |-- respiratory system diagnosis: long (nullable = true)
 |-- tracheostomy: long (nullable = true)
 |-- respiratory infection: long (nullable = true)
 |-- Total: long (nullable = true)



In [17]:
#from pyspark.sql.functions import substring, avg, sum, max, min, count

#air_quality=air_quality.filter(("Name == 'Ozone (O3)'"))

#air_quality=air_quality.groupBy("Borough","Year") \
 #       .agg(sum(air_quality.Data_Value).cast("INT").alias("sum_Data_Value"), \
  #       avg(air_quality.Data_Value).cast("INT").alias("avg_Data_Value"),
   #      max(air_quality.Data_Value).cast("INT").alias("max_Data_Value"),
    #     min(air_quality.Data_Value).cast("INT").alias("min_Data_Value")) 

In [18]:
# create gold database
spark.sql(
   """
   CREATE DATABASE IF NOT EXISTS PL7G1_GOLD LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/'
    """
)

DataFrame[]

In [19]:
#create table in gold
spark.sql(
   """
   DROP TABLE IF EXISTS PL7G1_GOLD.doencas
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_GOLD.doencas (
        borough string,
        year int,
        asthma long,
        bronchiolitis long,
        chronic_pulmonary_disease long,
        cystic_fibrosis long,
        ecmo long,
        alveolar_lung_diseases long,
        pneumonia long,
        pulmonary_embolism long,
        respiratory_failure long,
        respiratory_malignancy long,
        bronchitis long,
        respiratory_system_diagnosis long,
        tracheostomy long,
        respiratory_infection long,
        total long
        
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/doencas'
    """
)




DataFrame[]

In [20]:
#Tirar os " " e meter ""
from pyspark.sql import functions as F
hospital = hospital.select([F.col(col).alias(col.replace(' ', '_')) for col in hospital.columns])
hospital.show()

+-------------+----+------+-------------+-------------------------+---------------+----+----------------------+---------+------------------+-------------------+----------------------+----------+----------------------------+------------+---------------------+-----+
|      borough|year|asthma|bronchiolitis|chronic_pulmonary_disease|cystic_fibrosis|ECMO|alveolar_lung_diseases|pneumonia|pulmonary_embolism|respiratory_failure|respiratory_malignancy|bronchitis|respiratory_system_diagnosis|tracheostomy|respiratory_infection|Total|
+-------------+----+------+-------------+-------------------------+---------------+----+----------------------+---------+------------------+-------------------+----------------------+----------+----------------------------+------------+---------------------+-----+
|        Bronx|2020|  2352|          379|                     1865|              4|   0|                   190|     2588|               530|                900|                   543|       313|           

In [21]:
hospital \
    .select ("borough","year","asthma","bronchiolitis","chronic_pulmonary_disease","cystic_fibrosis","ecmo", "alveolar_lung_diseases", "pneumonia", "pulmonary_embolism", "respiratory_failure", "respiratory_malignancy", "bronchitis", "respiratory_system_diagnosis", "tracheostomy", "respiratory_infection", "total") \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/doencas")

In [22]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/doencas`
""").show()

++
||
++
++



In [23]:
# create table presto in gold
spark.sql(
    """
    DROP TABLE IF EXISTS PL7G1_GOLD.doencasPresto
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_GOLD.doencasPresto (
        borough string,
        year int,
        asthma long,
        bronchiolitis long,
        chronic_pulmonary_disease long,
        cystic_fibrosis long,
        ecmo long,
        alveolar_lung_diseases long,
        pneumonia long,
        pulmonary_embolism long,
        respiratory_failure long,
        respiratory_malignancy long,
        bronchitis long,
        respiratory_system_diagnosis long,
        tracheostomy long,
        respiratory_infection long,
        total long
    )
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/doencas/_symlink_format_manifest/'
"""
).show()


++
||
++
++

